# Setup

In [254]:
import pandas as pd
import os
import numpy as np
import json


In [255]:
data = pd.read_parquet(".." + os.sep + "data" + os.sep +
                "dati_presenze_politici.parquet")


trasformazione delle date in oggetti datetime

In [256]:
#data['DATA'] = data['DATA'].apply(lambda x: x.replace(" 00:00",""))
#data['DATA'] = data['DATA'].apply(lambda x: x.replace(":00",""))
#data.DATA = pd.to_datetime(data["DATA"],  format="%d.%m.%Y")


In [257]:
data.head(4)

,CANALE,PROGRAMMA,DATA,COGNOME,NOME,MICRO_CATEGORIA,ARGOMENTO,DURATA,TIPO_TEMPO
2,CANALE 5,TG5,2022-01-01,Berlusconi,Silvio,PDL - Forza Italia,Politica e attività istituzionali,42,Notizia
3,CANALE 5,TG5,2022-01-01,Calenda,Carlo,Azione - +Europa - Radicali Italiani,Politica e attività istituzionali,6,Notizia
4,CANALE 5,TG5,2022-01-01,Conte,Giuseppe,Movimento 5 Stelle,Politica e attività istituzionali,36,Notizia
5,CANALE 5,TG5,2022-01-01,D'Amato,Alessio,Partito Democratico,Cronaca,3,Notizia


In [258]:
data.columns

Index(['CANALE', 'PROGRAMMA', 'DATA', 'COGNOME', 'NOME', 'MICRO_CATEGORIA',
       'ARGOMENTO', 'DURATA', 'TIPO_TEMPO'],
      dtype='object')

In [259]:
data.MICRO_CATEGORIA.unique()

array(['PDL - Forza Italia', 'Azione - +Europa - Radicali Italiani',
       'Movimento 5 Stelle', 'Partito Democratico', 'Liberi e Uguali',
       'Coraggio Italia-Idea-Cambiamo!-Europeisti-Noi Di Centro (Noi Campani)',
       "Fratelli d'Italia", 'Italia Viva', 'Lega Nord', 'Altro',
       'Per le Autonomie - Minoranze linguistiche', 'LAlternativa',
       "Noi con l'Italia -  USEI - Rinascimento - AdC",
       'MAIE-Psi-Facciamo Eco', 'Centro Democratico',
       'Italia al Centro-Coraggio Italia', 'Europa Verde - Verdi Europei',
       'Manifesta, Potere al Popolo, Partito della Rifondazione Comunista-Sinistra Europea',
       'C.A.L. (Costituzione, Ambiente, Lavoro)-Alternativa-Pc-Idv',
       'Insieme per il futuro', 'Coraggio Italia-Idea e Cambiamo',
       "L'Alternativa c'è!", 'Europa Verde', 'Partito Liberale Europeo',
       'Partito Comunista', 'Potere al popolo',
       'Il Popolo della famiglia', 'Partito Comunista dei lavoratori',
       'Verità', 'Partito gay LGBT+', 'PC

In [260]:
def data2json(data, outfile, outpath=".." + os.sep + "docs" + os.sep + "data" + os.sep):
    json_data = json.dumps(data)
    json_file = open(outpath+outfile, "w")
    json_file.write(json_data)
    json_file.close()



In [388]:
data[(data.DATA >= '2022-01-01') & (data.DATA <= '2022-01-02')]

,CANALE,PROGRAMMA,DATA,COGNOME,NOME,MICRO_CATEGORIA,ARGOMENTO,DURATA,TIPO_TEMPO
2,CANALE 5,TG5,2022-01-01,Berlusconi,Silvio,PDL - Forza Italia,Politica e attività istituzionali,42,Notizia
3,CANALE 5,TG5,2022-01-01,Calenda,Carlo,Azione - +Europa - Radicali Italiani,Politica e attività istituzionali,6,Notizia
4,CANALE 5,TG5,2022-01-01,Conte,Giuseppe,Movimento 5 Stelle,Politica e attività istituzionali,36,Notizia
5,CANALE 5,TG5,2022-01-01,D'Amato,Alessio,Partito Democratico,Cronaca,3,Notizia
6,CANALE 5,TG5,2022-01-01,D'Amato,Alessio,Partito Democratico,Cronaca,24,Parola
...,...,...,...,...,...,...,...,...,...
19515,TV8,SKY TG24,2022-01-01,Renzi,Matteo,Italia Viva,Politica e attività istituzionali,40,Notizia
19516,TV8,SKY TG24,2022-01-02,Lega Nord,Soggetto Collettivo,Lega Nord,Politica e attività istituzionali,31,Notizia
19517,TV8,SKY TG24,2022-01-02,Meloni,Giorgia,Fratelli d'Italia,Politica e attività istituzionali,9,Notizia
19518,TV8,SKY TG24,2022-01-02,PDL - Forza Italia,Soggetto Collettivo,PDL - Forza Italia,Politica e attività istituzionali,7,Notizia


In [395]:
from datetime import datetime
date_time_obj = datetime.strptime("01/01/2022", '%d/%m/%Y')


In [396]:
data[(data.DATA >= date_time_obj) & (data.DATA <= date_time_obj)]


,CANALE,PROGRAMMA,DATA,COGNOME,NOME,MICRO_CATEGORIA,ARGOMENTO,DURATA,TIPO_TEMPO
2,CANALE 5,TG5,2022-01-01,Berlusconi,Silvio,PDL - Forza Italia,Politica e attività istituzionali,42,Notizia
3,CANALE 5,TG5,2022-01-01,Calenda,Carlo,Azione - +Europa - Radicali Italiani,Politica e attività istituzionali,6,Notizia
4,CANALE 5,TG5,2022-01-01,Conte,Giuseppe,Movimento 5 Stelle,Politica e attività istituzionali,36,Notizia
5,CANALE 5,TG5,2022-01-01,D'Amato,Alessio,Partito Democratico,Cronaca,3,Notizia
6,CANALE 5,TG5,2022-01-01,D'Amato,Alessio,Partito Democratico,Cronaca,24,Parola
...,...,...,...,...,...,...,...,...,...
19511,TV8,SKY TG24,2022-01-01,Gelmini,Mariastella,PDL - Forza Italia,Politica e attività istituzionali,25,Notizia
19512,TV8,SKY TG24,2022-01-01,Lega Nord,Soggetto Collettivo,Lega Nord,Politica e attività istituzionali,13,Notizia
19513,TV8,SKY TG24,2022-01-01,Letta,Enrico,Partito Democratico,Politica e attività istituzionali,37,Notizia
19514,TV8,SKY TG24,2022-01-01,Meloni,Giorgia,Fratelli d'Italia,Politica e attività istituzionali,43,Notizia


# Soggetto collettivo

filtro per soggetti collettivo

In [261]:
dati_soggetti_colletivi = data[data.NOME =="Soggetto Collettivo"]
#dati_soggetti_colletivi = dati_soggetti_colletivi[dati_soggetti_colletivi.NOME ==
#                                        "Soggettivo Collettivo"]

In [262]:
elenco_soggetti_collettivi =  list(dati_soggetti_colletivi.COGNOME.unique())


In [263]:
categorie_argomento = list(data.ARGOMENTO.unique())


## Distribuzione Interventi ed argomenti per Soggetti Collettivi

In [264]:
minuti_soggetti_collettivi = dati_soggetti_colletivi.groupby(["COGNOME"])["DURATA"].sum().reset_index().fillna(0).sort_values(by="DURATA", ascending=False)

In [265]:
categorie_argomento = list(dati_soggetti_colletivi.ARGOMENTO.unique())

In [266]:
color18 = [
    '#38b6b6', #1
    '#d5b158', #2
    '#3cb371', #3
    '#ff6666', #4
    '#6699ff', #5
    '#eaf889', #6
    '#0a915d', #7
    '#a092f1', #8 
    '#ff6347', #9
    '#82b6e9', #10
    '#2e4783', #11
    '#dcc392', #12
    '#d3758f', #13
    '#f3d999', #14
    '#a4d8c2', #15
    '#c6b38e',  #16
    '#b8d2c7', #17
    '#f5e8c8' #18
]

In [267]:
coloriCategorie = {}
for categoria in categorie_argomento:
    coloriCategorie[categoria] = color18[categorie_argomento.index(
        categoria)]


## Volume ore per soggetto solo argomento politica

In [268]:
solo_politica = dati_soggetti_colletivi[dati_soggetti_colletivi.ARGOMENTO ==
                        "Politica e attività istituzionali"]
minuti_soggetti_collettivi_solo_politica = solo_politica.groupby(["COGNOME"])[
    "DURATA"].sum().reset_index().fillna(0).sort_values(["DURATA","COGNOME"], ascending=[True,True])

In [269]:
volume_politica = {}
nomi = []
valori = []
for idx, row in minuti_soggetti_collettivi_solo_politica.iterrows():
    nomi.append(row.COGNOME)
    valori.append(round(row.DURATA/60, 2))
volume_politica['categorie'] = nomi
volume_politica['valori'] = valori  
volume_politica['titolo'] = "Volume in ore interventi del tema politica e attività istituzionali"
volume_politica['etichetta'] = 'Politica e Attività Istituzionali'
data2json(volume_politica,"interventi_solo_politica.json")

## Treemap presenza movimenti

In [270]:
data_distro_interventi = {}
data_argomenti_movimenti = {}
oggetti_distro_interventi = []
label_notizia = "notizia"
label_intervento = "intervento"
for idx,row in minuti_soggetti_collettivi.iterrows():
    root = {}
    name = row.COGNOME
    root['name'] = name
    value = round(row.DURATA / 60,2)   
    root['value'] = value
    label = {}
    label['show'] = 'true'
    formatter = name + "\n" + str(value) + " ore\n"
    label['formatter'] = formatter
    root['label'] = label
    itemStyle = {}
    itemStyle['borderWidth'] =1
    itemStyle['borderColor'] = '#000000'
    itemStyle['opacity'] = 0.5
    root['itemStyle'] = itemStyle
    oggetti_distro_interventi.append(root)
    data_distro_interventi["dati"] = oggetti_distro_interventi
    
data2json(data_distro_interventi, "presenza_movimenti.json")

## Flussi interventi no politica

In [271]:
senza_politica = dati_soggetti_colletivi[dati_soggetti_colletivi.ARGOMENTO !=
                                         "Politica e attività istituzionali"]
minuti_soggetti_collettivi_senza_politica = senza_politica.groupby(["COGNOME"])[
    "DURATA"].sum().reset_index().fillna(0).sort_values(["DURATA", "COGNOME"], ascending=[True, True])
presenze_senza_poitica = senza_politica.pivot_table(
    index="COGNOME", values="DURATA", aggfunc=np.sum)
limite10 = presenze_senza_poitica.DURATA.quantile(0.9)

In [272]:
categorie_durata = senza_politica.pivot_table(index='ARGOMENTO',values="DURATA", aggfunc=np.sum).reset_index()


In [273]:
categorie_durata


,ARGOMENTO,DURATA
0,Altro,33874
1,Ambiente,250
2,Costume e società,31
3,Cronaca,47003
4,Cultura e istruzione,2018
5,"Economia, finanza e lavoro",52839
6,Esteri,3914
7,Fine,6
8,Giustizia,5700
9,Informazioni di servizio,237


In [274]:
categorie_durata['DIMENSIONE'] = categorie_durata['DURATA'].apply(
    lambda x: round((x / categorie_durata.DURATA.sum() * 100), 1))

In [275]:
categorie_durata


,ARGOMENTO,DURATA,DIMENSIONE
0,Altro,33874,21.5
1,Ambiente,250,0.2
2,Costume e società,31,0.0
3,Cronaca,47003,29.9
4,Cultura e istruzione,2018,1.3
5,"Economia, finanza e lavoro",52839,33.6
6,Esteri,3914,2.5
7,Fine,6,0.0
8,Giustizia,5700,3.6
9,Informazioni di servizio,237,0.2


In [276]:
categorie_durata['DIMENSIONE'] = categorie_durata['DIMENSIONE'].apply(lambda x: x+3) #if x <= 0 else x)


In [277]:
senza_politica


,CANALE,PROGRAMMA,DATA,COGNOME,NOME,MICRO_CATEGORIA,ARGOMENTO,DURATA,TIPO_TEMPO
71,CANALE 5,TG5,2022-01-04,Lega Nord,Soggetto Collettivo,Lega Nord,"Economia, finanza e lavoro",2,Notizia
72,CANALE 5,TG5,2022-01-04,Lega Nord,Soggetto Collettivo,Lega Nord,"Medicina, salute e scienza",3,Notizia
74,CANALE 5,TG5,2022-01-04,Movimento 5 Stelle,Soggetto Collettivo,Movimento 5 Stelle,"Economia, finanza e lavoro",2,Notizia
75,CANALE 5,TG5,2022-01-04,Movimento 5 Stelle,Soggetto Collettivo,Movimento 5 Stelle,"Medicina, salute e scienza",3,Notizia
97,CANALE 5,TG5,2022-01-05,Partito Democratico,Soggetto Collettivo,Partito Democratico,Cultura e istruzione,3,Notizia
...,...,...,...,...,...,...,...,...,...
26860,RETE 4,TG4,2021-01-31,Partito Democratico,Soggetto Collettivo,Partito Democratico,"Economia, finanza e lavoro",5,Notizia
29526,RAINEWS,RAI NEWS,2021-01-28,Lega Nord,Soggetto Collettivo,Lega Nord,Cronaca,4,Notizia
29977,SKY TG 24 CAN. 50,SKY TG24,2021-01-07,Lega Nord,Soggetto Collettivo,Lega Nord,Esteri,9,Notizia
30204,SKY TG24,SKY TG24,2021-01-03,Lega Nord,Soggetto Collettivo,Lega Nord,Cronaca,2,Notizia


In [278]:
sankey_data_name = []
sankey_data_links = []
sankey_data = {}
sankey_data_categorie = []

for idx, row in categorie_durata.iterrows():
    categoria = row.ARGOMENTO
    name_sankey = {}
    name_sankey['name'] = categoria
    name_sankey['symbolSize'] = row.DIMENSIONE
    name_sankey['value'] = row.DURATA/60
    name_sankey['category'] = categoria
    sankey_data_name.append(name_sankey)
    category = {}
    category['name'] = categoria
    itemstyle = {}
    itemstyle['color'] = coloriCategorie[categoria]
    category['itemStyle'] = itemstyle
    sankey_data_categorie.append(category)

category = {}
category['name'] = "Soggetto politico"
itemstyle = {}
itemstyle['color'] = "gray"
category['itemStyle'] = itemstyle
sankey_data_categorie.append(category)

   
for nome in senza_politica.COGNOME.unique():
    name_sankey = {}
    name_sankey['name'] = nome
    name_sankey['category'] = "Soggetto politico"
    name_sankey['symbolSize'] = 20
    sankey_data_name.append(name_sankey)
    senza_politica_gruppo = senza_politica[senza_politica.COGNOME == nome]
    argomenti = senza_politica_gruppo.groupby(["ARGOMENTO"])["DURATA"].sum(
    ).reset_index().fillna(0).sort_values(by="DURATA", ascending=False)
    for idx, row in argomenti.iterrows():
        oggetto = {}
        oggetto['source'] = row.ARGOMENTO
        oggetto['target'] = nome
        oggetto['value'] = round(row.DURATA / 60, 2)
        itemstyle = {}
        itemstyle['color'] = coloriCategorie[row.ARGOMENTO]
        oggetto['itemStyle'] = itemstyle
        sankey_data_links.append(oggetto)
sankey_data['data'] = sankey_data_name
sankey_data['links'] = sankey_data_links
sankey_data['categories'] = sankey_data_categorie
data2json(sankey_data, "argomenti_movimenti.json")

In [279]:
argomenti

,ARGOMENTO,DURATA
0,"Economia, finanza e lavoro",1


# Personaggi politici

filtro per soli politici

In [280]:
dati_politici = data[data.NOME != "Soggettivo Collettivo"]
dati_politici = dati_politici[dati_politici.NOME != "Soggetto Collettivo"]

creazione campo NOME_COGNOME

In [281]:
dati_politici['NOME_COGNOME'] = dati_politici['NOME'] + \
    " " + dati_politici['COGNOME']


In [282]:
elenco_politici = list(dati_politici['NOME_COGNOME'].unique())

In [283]:
elenco_politici


['Silvio Berlusconi',
 'Carlo Calenda',
 'Giuseppe Conte',
 "Alessio D'Amato",
 'Nicola Fratoianni',
 'Enrico Letta',
 'Clemente Mastella',
 'Giorgia Meloni',
 'Matteo Renzi',
 'Matteo Salvini',
 'Antonio Tajani',
 'Davide Crippa',
 "Massimo D'Alema",
 'Paola Pula',
 'Pier Luigi Lopalco',
 'Enrico Borghi',
 'Wanda Ferro',
 'Vincenzo De Luca',
 'Massimiliano Fedriga',
 'Federico Monti',
 'Lorena Vedovato',
 'Luciano Nobili',
 'Stefano Bonaccini',
 'Roberto Occhiuto',
 'Simone Pillitteri',
 'Chiara Braga',
 'Federico Sboarina',
 'Stefano Ceccanti',
 'Sara Cunial',
 'Giancarlo Giorgetti',
 'Carlo Bagnasco',
 'Beppe Grillo',
 'Raffaele Lombardo',
 'Nicola Zingaretti',
 'Nello Musumeci',
 'Alessandro Cattaneo',
 'Michele Emiliano',
 'Lorenzo Fontana',
 'Eugenio Giani',
 'Debora Serracchiani',
 'Giovanni Toti',
 'Luca Zaia',
 'Santina Lattuca',
 'Virginia Raggi',
 'Licia Ronzulli',
 'Vito Crimi',
 'Walter Ricciardi',
 'Antonio Del Giudice',
 'Anna Rossomando',
 'Dario Franceschini',
 'Paolo 

calcolo presenze per politico

In [284]:
presenze = dati_politici.pivot_table(index="NOME_COGNOME",
                             values="DURATA", aggfunc=np.sum)#

In [285]:
presenze.reset_index(inplace=True)


In [286]:
presenze.sort_values(by="DURATA", ascending=False, inplace=True)

In [287]:
totale_politici = len(dati_politici.NOME_COGNOME.unique())


In [288]:
totale_politici


8547

ricerca più presenti nell'ultimo decile

In [289]:
#ultimo decile
p = presenze.DURATA.quantile(0.9)

In [290]:
presenze_politici_rilevanti = presenze[(presenze.DURATA > presenze.DURATA.quantile(0.9)) & (presenze.DURATA <= presenze.DURATA.max())]


di questi prendiamo l'ultimo decile

In [291]:
presenze_politici_top_rilevanti = presenze[(presenze.DURATA > presenze_politici_rilevanti.DURATA.quantile(
    0.9)) & (presenze.DURATA <= presenze.DURATA.max())]


In [292]:
presenze_politici_top_rilevanti.shape[0]

86

In [293]:
presenze_politici_top_rilevanti

,NOME_COGNOME,DURATA
5887,Matteo Salvini,4830594
7830,Silvio Berlusconi,2542977
5001,Luigi Di Maio,2460683
5880,Matteo Renzi,1896863
3719,Giorgia Meloni,1835590
...,...,...
6929,Pietro Grasso,167791
5412,Maria Elena Boschi,165889
3641,Gianni Cuperlo,165194
4933,Luciano Nobili,160188


# Distribuzione argomenti

In [294]:
distribuzione_argomenti = data.groupby(['DATA', 'ARGOMENTO','TIPO_TEMPO'])[
    'DURATA'].aggregate('sum').unstack().reset_index()

In [295]:
distribuzione_argomenti.fillna(0, inplace=True)

In [296]:
distribuzione_argomenti['TOTALE'] = distribuzione_argomenti['Notizia'] + distribuzione_argomenti['Parola']

In [297]:
distribuzione_argomenti


TIPO_TEMPO,DATA,ARGOMENTO,Notizia,Parola,TOTALE
0,2018-01-01,Costume e società,16.0,0.0,16.0
1,2018-01-01,Cronaca,154.0,404.0,558.0
2,2018-01-01,"Economia, finanza e lavoro",11.0,0.0,11.0
3,2018-01-01,Giustizia,261.0,247.0,508.0
4,2018-01-01,Mass media e spettacolo,0.0,46.0,46.0
...,...,...,...,...,...
13197,2022-06-30,Costume e società,0.0,290.0,290.0
13198,2022-06-30,Cronaca,20.0,159.0,179.0
13199,2022-06-30,"Economia, finanza e lavoro",188.0,2706.0,2894.0
13200,2022-06-30,Esteri,109.0,1226.0,1335.0


In [298]:
dati = []
radar_dati = {}


In [299]:

totale_parole = distribuzione_argomenti.sort_values(by="ARGOMENTO", ascending=True).groupby(
    'ARGOMENTO')['Parola'].aggregate('sum').to_frame().reset_index()
dati_interventi = {}
label_interventi = "Interventi"
interventi_argomenti_dati = []
for idx, row in totale_parole.iterrows():
    v = round(row.Parola / 60, 2)
    interventi_argomenti_dati.append(v)
dati_interventi['value'] = interventi_argomenti_dati
dati_interventi['name'] = label_interventi
dati.append(dati_interventi)


In [300]:
totale_notizie = distribuzione_argomenti.sort_values(by="ARGOMENTO", ascending=True).groupby(
    'ARGOMENTO')['Notizia'].aggregate('sum').to_frame().reset_index()

dati_notizie = {}
label_notizie = "Notizie"
notizie_argomenti_dati = []
out = "["
for idx, row in totale_notizie.iterrows():
    v = round(row.Notizia / 60,2)
    out += str(v) + ","
    notizie_argomenti_dati.append(v)
out = out[:-1] + "]"
dati_notizie['value'] = notizie_argomenti_dati
dati_notizie['name']=label_notizie
dati.append(dati_notizie)


In [301]:
minuti_argomenti = distribuzione_argomenti.pivot_table(index="ARGOMENTO", values="TOTALE", aggfunc=np.sum).reset_index().sort_values(by="ARGOMENTO", ascending=True)

In [302]:
indicators = []
for idx, row in minuti_argomenti.iterrows():
    blocco = {}
    name = row.ARGOMENTO
    value = round(row.TOTALE / 60,2)
    blocco['name'] = name 
    blocco['value'] = value
    out += "{ name: '%s', value: %s}," % (name, value)
    indicators.append(blocco)

In [303]:
radar_dati = {}
radar_dati['titolo'] = "Distribuzione per argomenti"
radar_dati['titolo_radar'] = "Interventi vs Notizie"
radar_dati['indicatori'] = indicators
radar_dati['legenda'] = [label_interventi, label_notizie]
radar_dati['dati'] = dati


In [304]:
data2json(radar_dati,"radar_argomenti.json")

# Flusso notizie/interventi nel tempo

In [305]:
label_interventi = "INTERVENTI"
label_notizie = "NOTIZIE"
legenda = []
legenda.append(label_interventi)
legenda.append(label_notizie)
volume_notizie = {}
volume_notizie['titolo'] = "Ore di notizie ed interventi di politica nella televisione"
volume_notizie['legenda'] = legenda
dati = []


In [306]:
volume_notizie_parole = data.groupby(['DATA', 'TIPO_TEMPO'])['DURATA'].aggregate('sum').unstack().reset_index()
volume_notizie_parole.sort_values(by="DATA", ascending=True, inplace=True)
volume_notizie_parole['TOTALE'] = volume_notizie_parole['Notizia'] + \
    volume_notizie_parole['Parola']


In [307]:
volume_notizie_parole[volume_notizie_parole.TOTALE ==
                      volume_notizie_parole.TOTALE.max()]


TIPO_TEMPO,DATA,Notizia,Parola,TOTALE
1114,2021-01-19,18186,163020,181206


In [308]:
for idx,row in volume_notizie_parole.iterrows():
    day = row.DATA.strftime('%Y/%m/%d')
    value_parole = []
    value_notizie = []
    parole = round(row.Parola / 60,2)
    notize = round(row.Notizia /60,2)
    value_parole.append(day)
    value_parole.append(parole)
    value_parole.append(label_interventi)
    value_notizie.append(day)
    value_notizie.append(notize)
    value_notizie.append(label_notizie)
    dati.append(value_parole)
    dati.append(value_notizie)


In [309]:
volume_notizie['dati'] = dati

In [310]:
data2json(volume_notizie, "volume_notizie.json")

# Test su un politico

In [383]:
politico = dati_politici[dati_politici['NOME_COGNOME'] == "Lorenzo Dellai"]

In [384]:
politico.shape[0]

0

In [367]:
politico.rename(columns={'CANALE': 'channel', 'PROGRAMMA': 'program', 'DATA':'day', 
                     'COGNOME':'lastname','NOME':'name',
                     'MICRO_CATEGORIA':'affiliation',
                         'ARGOMENTO': 'topic', 'DURATA': 'minutes_of_intervention',
                     'TIPO_TEMPO':'category_intervention'}, inplace=True)

In [368]:
exclude = ['name', 'lastname', 'name_lastname']
a = set(politico.columns)
b = set(exclude)
selected_columns = list(a.difference(b))

In [353]:
#politico[selected_columns].to_dict('records')


In [369]:
fromdate = politico.day.min().strftime('%d/%m/%Y')
todata = politico.day.max().strftime('%d/%m/%Y')
politico.day = politico.day.dt.strftime('%d/%m/%Y')


In [370]:

presence = politico.minutes_of_intervention.sum()

In [371]:
presence


1964

In [372]:
topic_total = politico.groupby('topic').aggregate('sum')

In [325]:
topic_total.rename(columns={"minutes_of_intervention":"time_pro_topics"},inplace=True)


In [326]:
topic_total.to_dict()


{'time_pro_topics': {'Altro': 1776,
  'Ambiente': 1857,
  'Costume e società': 6163,
  'Cronaca': 40090,
  'Cultura e istruzione': 47639,
  'Economia, finanza e lavoro': 20269,
  'Esteri': 16874,
  'Giustizia': 2470,
  'Informazioni di servizio': 142,
  'Mass media e spettacolo': 11109,
  'Medicina, salute e scienza': 7557,
  'Politica e attività istituzionali': 352590,
  'Religione e questioni religiose': 165,
  'Società': 12705,
  'Sport': 1021}}

In [379]:
time_channels = politico.groupby('channel').aggregate('sum')
time_channels.rename(
    columns={"minutes_of_intervention": "time_pro_channel"}, inplace=True)


In [380]:
time_channels


,time_pro_channel
channel,
CANALE 5,224
ITALIA 1,75
LA7,12
Mediaset TgCom 24,670
RAI 1,13
RAI 2,9
RAI 3,59
RAINEWS,752
RETE 4,112


In [320]:
categorie_durata.to_dict('records')


[{'ARGOMENTO': 'Altro', 'DURATA': 33874, 'DIMENSIONE': 24.5},
 {'ARGOMENTO': 'Ambiente', 'DURATA': 250, 'DIMENSIONE': 3.2},
 {'ARGOMENTO': 'Costume e società', 'DURATA': 31, 'DIMENSIONE': 3.0},
 {'ARGOMENTO': 'Cronaca', 'DURATA': 47003, 'DIMENSIONE': 32.9},
 {'ARGOMENTO': 'Cultura e istruzione', 'DURATA': 2018, 'DIMENSIONE': 4.3},
 {'ARGOMENTO': 'Economia, finanza e lavoro',
  'DURATA': 52839,
  'DIMENSIONE': 36.6},
 {'ARGOMENTO': 'Esteri', 'DURATA': 3914, 'DIMENSIONE': 5.5},
 {'ARGOMENTO': 'Fine', 'DURATA': 6, 'DIMENSIONE': 3.0},
 {'ARGOMENTO': 'Giustizia', 'DURATA': 5700, 'DIMENSIONE': 6.6},
 {'ARGOMENTO': 'Informazioni di servizio', 'DURATA': 237, 'DIMENSIONE': 3.2},
 {'ARGOMENTO': 'Mass media e spettacolo', 'DURATA': 1313, 'DIMENSIONE': 3.8},
 {'ARGOMENTO': 'Medicina, salute e scienza',
  'DURATA': 3119,
  'DIMENSIONE': 5.0},
 {'ARGOMENTO': 'Programma', 'DURATA': 195, 'DIMENSIONE': 3.1},
 {'ARGOMENTO': 'Pubblicità', 'DURATA': 32, 'DIMENSIONE': 3.0},
 {'ARGOMENTO': 'Religione e ques

In [321]:
politico


,channel,program,day,lastname,name,affiliation,topic,minutes_of_intervention,category_intervention,NOME_COGNOME
490,CANALE 5,TG5,20/01/2022,Sgarbi,Vittorio,Noi con l'Italia - USEI - Rinascimento - AdC,Cultura e istruzione,127,Parola,Vittorio Sgarbi
818,CANALE 5,TG5,28/01/2022,Sgarbi,Vittorio,Noi con l'Italia - USEI - Rinascimento - AdC,Politica e attività istituzionali,1,Notizia,Vittorio Sgarbi
1157,CIELO,SKY TG24 - BUONGIORNO,20/01/2022,Sgarbi,Vittorio,Noi con l'Italia - USEI - Rinascimento - AdC,Politica e attività istituzionali,71,Parola,Vittorio Sgarbi
1250,ITALIA 1,STUDIO APERTO,01/01/2022,Sgarbi,Vittorio,Noi con l'Italia - USEI - Rinascimento - AdC,Cultura e istruzione,11,Parola,Vittorio Sgarbi
2792,LA7,TG LA7,12/01/2022,Sgarbi,Vittorio,Noi con l'Italia - USEI - Rinascimento - AdC,Politica e attività istituzionali,5,Notizia,Vittorio Sgarbi
...,...,...,...,...,...,...,...,...,...,...
28545,SKY TG24,SKY TG24 - AGENDA,24/01/2021,Sgarbi,Vittorio,l'Italia,Politica e attività istituzionali,27,Parola,Vittorio Sgarbi
29699,RETE 4,QUARTA REPUBBLICA SPECIALE,19/01/2021,Sgarbi,Vittorio,l'Italia,Politica e attività istituzionali,573,Parola,Vittorio Sgarbi
29744,RETE 4,STASERA ITALIA SPECIALE (VN),06/01/2021,Sgarbi,Vittorio,l'Italia,Politica e attività istituzionali,463,Parola,Vittorio Sgarbi
29748,RETE 4,STASERA ITALIA SPECIALE (VN),27/01/2021,Sgarbi,Vittorio,l'Italia,Cronaca,305,Parola,Vittorio Sgarbi
